# TruBot

Example setup and monitoring of a conversational bot with context made up of the
TruEra website. This example requires either a pinecone vector db set up with
some contexts to answer questions with or alternatively can use the local
database for use with hnswlib provided here. To use hnswlib, some additional
requirements need to be installed with pip. Regardless of the vector db
provider, the example feedback functions here use openai and huggingface free
inference APIs and need their respective keys to be provided in a .env file.

## HNSWLIB additional requirements

Run the following in your shell or the equivalent in the following cell to
install additional requirements for use with HNSWLIB. This is not required if
you are running this example with a pinecone db.

```bash
pip install docarray hnswlib
```

In [1]:
# ! pip install docarray hnswlib

In [2]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""
None

## API keys setup

In [3]:
from trulens_eval.keys import check_keys

check_keys(
    "OPENAI_API_KEY",
    "HUGGINGFACE_API_KEY"
)

✅ Key OPENAI_API_KEY set from environment (same value found in .env file at /Users/piotrm/Dropbox/repos/github/trulens/.env).
✅ Key HUGGINGFACE_API_KEY set from environment (same value found in .env file at /Users/piotrm/Dropbox/repos/github/trulens/.env).


In [4]:
import os
from pprint import PrettyPrinter

# Imports from langchain to build app:
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryBufferMemory
import numpy as np

# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import feedback
from trulens_eval import FeedbackMode
from trulens_eval import Select
from trulens_eval import TP
from trulens_eval import Tru
from trulens_eval.utils.langchain import WithFeedbackFilterDocuments

pp = PrettyPrinter()

# Tru object manages the database of apps, records, and feedbacks; and the
# dashboard to display these.
tru = Tru()

# Start the dasshboard. If you running from github repo, you will need to adjust
# the path the dashboard streamlit app starts in by providing the _dev argument.
tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

# If needed, you can reset the trulens_eval dashboard database by running the
# below line:

# tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
Force stopping dashboard ...
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.86.40:8501 .


In [5]:
# Select vector db provider. Pinecone requires setting up a pinecone database
# first while the hnsw database is included with trulens_eval.
# db_host = "pinecone"
db_host = "hnsw"

model_name = "gpt-3.5-turbo"
app_id = "TruBot"

# Embedding for vector db.
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims

if db_host == "pinecone":
    check_keys(
        "PINECONE_API_KEY",
        "PINECONE_ENV"
    )

    # Pinecone configuration if using pinecone.

    from langchain.vectorstores import Pinecone
    import pinecone

    pinecone.init(
        api_key=os.environ.get("PINECONE_API_KEY"),  # find at app.pinecone.io
        environment=os.environ.get("PINECONE_ENV")  # next to api key in console
    )

    # If using pinecone, make sure you create your index under name 'llmdemo' or
    # change the below.

    def get_doc_search():

        docsearch = Pinecone.from_existing_index(
            index_name="llmdemo", embedding=embedding
        )

        return docsearch

elif db_host == "hnsw":
    # Local pinecone alternative. Requires precomputed 'hnswlib_truera' folder.

    from langchain.vectorstores import DocArrayHnswSearch

    def get_doc_search():
        # We need to create this object in the thread in which it is used so we
        # wrap it in this function for later usage.

        docsearch = DocArrayHnswSearch.from_params(
            embedding=embedding,
            work_dir='hnswlib_trubot',
            n_dim=1536,
            max_elements=1024
        )

        return docsearch
else:
    raise RuntimeError("Unhandled db_host, select either 'pinecone' or 'hnsw'.")

# LLM for completing prompts, and other tasks.
llm = OpenAI(temperature=0, max_tokens=256)

# Construct feedback functions.

# API endpoints for models used in feedback functions:
hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on_input_output()
# By default this will evaluate feedback on main app input and main app output.

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()
# By default this will evaluate feedback on main app input and main app output.

# Question/statement relevance between question and each context chunk.
f_qs_relevance = feedback.Feedback(openai.qs_relevance).on_input().on(
    Select.Record.app.combine_docs_chain._call.args.inputs.input_documents[:].page_content
).aggregate(np.min)
# First feedback argument is set to main app input, and the second is taken from
# the context sources as passed to an internal `combine_docs_chain._call`.

all_feedbacks = [f_lang_match, f_qa_relevance, f_qs_relevance]

✅ In language_match, input text1 will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In language_match, input text2 will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to *.__record__.app.combine_docs_chain._call.args.inputs.input_documents[:].page_content .


# TruBot Version 1

In [6]:
def v1_new_conversation(feedback_mode=FeedbackMode.WITH_APP):
    """
    Create a langchain app for a new conversation with a question-answering bot.

    Feedback_mode controls when feedback is evaluated:

    - FeedbackMode.WITH_APP -- app will wait until feedback is evaluated before
      returning from calls.

    - FeedbackMode.WITH_APP_THREAD -- app will return from calls and evaluate
      feedback in a new thread.

    - FeedbackMode.DEFERRED -- app will return and a separate runner thread (see
      usage later in this notebook) will evaluate feedback.
    """

    # Blank conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=650,
        llm=llm,
        memory_key="chat_history",
        output_key='answer'
    )

    docsearch = get_doc_search()

    # Context retriever.
    retriever = docsearch.as_retriever()

    # Conversational app puts it all together.
    app = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        memory=memory,
        get_chat_history=lambda a: a,
        max_tokens_limit=4096
    )

    # Trulens instrumentation.
    tc = Tru().Chain(
        app_id=f"{app_id}/v1",
        chain=app,
        feedbacks=all_feedbacks,
        feedback_mode=feedback_mode, 
    )

    return app, tc

In [10]:
# Instantiate the app with fresh memory:

app1, tc1 = v1_new_conversation()

# Call the app:

res, record = tc1.call_with_record("Who is Shayak?")
res

# Notice the `source_documents` returned include chunks about Shameek and the
# answer includes bits about Shameek as a result.

creating new recording context
hello <function Chain.__call__ at 0x134701430>
hello <function BaseConversationalRetrievalChain._call at 0x135270ee0>
hello <function VectorStoreRetriever._get_relevant_documents at 0x127f59280>


Unsure what the main input string is for the call to _get_relevant_documents with args ['Who is Shayak?', <langchain.callbacks.manager.CallbackManagerForRetrieverRun object at 0x6b1f3a820>].
Unsure what the main output string is for the call to _get_relevant_documents.
Error calling wrapped function _call.
Traceback (most recent call last):
  File "/Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/trulens_eval/instruments.py", line 789, in wrapper
    rets, cost = Endpoint.track_all_costs_tally(
  File "/Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/trulens_eval/feedback/provider/endpoint/base.py", line 331, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
  File "/Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/trulens_eval/feedback/provider/endpoint/base.py", line 274, in track_all_costs
    return Endpoint._track_costs(thunk, with_endpoints=endpoints)
  File "/Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/trulens_eval/feedback/provi

_on_add_record, bindings=<BoundArguments (self=VectorStoreRetriever(tags=['DocArrayHnswSearch'], metadata=None, vectorstore=<langchain.vectorstores.docarray.hnsw.DocArrayHnswSearch object at 0x29f99e280>, search_type='similarity', search_kwargs={}), query='Who is Shayak?', run_manager=<langchain.callbacks.manager.CallbackManagerForRetrieverRun object at 0x6b1f3a820>)>
adding a record
finishing context with 0 records.


UnboundLocalError: local variable 'cost' referenced before assignment

In [8]:
record.dict()

{'record_id': 'record_hash_51c35ce87e4c0bcb312ca7ffd1f269d8',
 'app_id': 'TruBot/v1',
 'cost': {'n_requests': 1,
  'n_successful_requests': 1,
  'n_classes': 0,
  'n_tokens': 5,
  'n_stream_chunks': 0,
  'n_prompt_tokens': 5,
  'n_completion_tokens': 0,
  'cost': 0.0},
 'perf': {'start_time': datetime.datetime(2023, 8, 22, 16, 12, 36, 526082),
  'end_time': datetime.datetime(2023, 8, 22, 16, 12, 36, 951764)},
 'ts': datetime.datetime(2023, 8, 22, 16, 12, 36, 952883),
 'tags': '-',
 'main_input': 'Who is Shayak?',
 'main_output': {'page_content': 'When Shayak started building production grade machine learning models for algorithmic trading 10 years ago, he realized the need for putting the ‘science’ back in ‘data science’. Since then, he has been building systems and leading research to make machine learning and big data systems more explainable, privacy compliant, and fair. Shayak’s research at Carnegie Mellon University introduced a number of pioneering breakthroughs to the field of e

In [9]:
app, tc = v1_new_conversation()
res_with, record_with = tc.with_record(app.__call__, "Who is Shayak?") # or app.__call__

creating new recording context
hello <function Chain.__call__ at 0x134701430>


A new object of type <class 'langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain'> at 0x29fa3b300 is calling an instrumented method <function Chain.__call__ at 0x134701430>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x295ca4340) using this function.
A new object of type <class 'langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain'> at 0x29fa3b300 is calling an instrumented method <function BaseConversationalRetrievalChain._call at 0x135270ee0>. The path of this call may be incorrect.
Guessing path of new object is *.app based on other object (0x295ca4340) using this function.


hello <function BaseConversationalRetrievalChain._call at 0x135270ee0>
hello <function VectorStoreRetriever._get_relevant_documents at 0x127f59280>


Unsure what the main input string is for the call to _get_relevant_documents with args ['Who is Shayak?', <langchain.callbacks.manager.CallbackManagerForRetrieverRun object at 0x29fa35d00>].
Unsure what the main output string is for the call to _get_relevant_documents.
Error calling wrapped function _call.
Traceback (most recent call last):
  File "/Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/trulens_eval/instruments.py", line 789, in wrapper
    rets, cost = Endpoint.track_all_costs_tally(
  File "/Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/trulens_eval/feedback/provider/endpoint/base.py", line 331, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
  File "/Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/trulens_eval/feedback/provider/endpoint/base.py", line 274, in track_all_costs
    return Endpoint._track_costs(thunk, with_endpoints=endpoints)
  File "/Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/trulens_eval/feedback/provi

_on_add_record, bindings=<BoundArguments (self=VectorStoreRetriever(tags=['DocArrayHnswSearch'], metadata=None, vectorstore=<langchain.vectorstores.docarray.hnsw.DocArrayHnswSearch object at 0x10762afa0>, search_type='similarity', search_kwargs={}), query='Who is Shayak?', run_manager=<langchain.callbacks.manager.CallbackManagerForRetrieverRun object at 0x29fa35d00>)>
adding a record
finishing context with 0 records.


UnboundLocalError: local variable 'cost' referenced before assignment

In [ ]:
app, tc = v1_new_conversation()
with tc as records:
    res_context = app("Who is Shayak?")  # should be same as res_with

record_context = records[0] # should be same as record_with

In [ ]:
# The feedback should already be present in the dashboard, but we can check the
# qs_relevance here manually as well:
feedback = f_qs_relevance.run(record=record, app=tc1)
feedback.dict()

In [ ]:
# Now a question about QII (quantitative input influence is a base technology
# employed in TruEra's products) question but in a non-English language:

# Start a new conversation as the app keeps prior questions in its memory which
# may cause you some testing woes.
tc1 = v1_new_conversation()

# res, record = tc1.call_with_record("Co jest QII?") # Polish
res, record = tc1.call_with_record("Was ist QII?") # German
res

# Note here the response is in English. This example sometimes matches language
# so other variants may need to be tested.

In [ ]:
# Language match failure can be seen using the f_lang_match (and is visible in
# dashboard):
feedback = f_lang_match.run(record=record, app=tc1)
feedback.dict()

# TruBot Version 2 - Language match fix

In [ ]:
def v2_new_conversation(feedback_mode=FeedbackMode.WITH_APP):
    """
    Create a langchain app for a new conversation with a question-answering bot.
    """

    # Blank conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=650,
        llm=llm,
        memory_key="chat_history",
        output_key='answer'
    )

    docsearch = get_doc_search()

    # Context retriever.
    retriever = docsearch.as_retriever()

    # Conversational app puts it all together.
    app = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        memory=memory,
        get_chat_history=lambda a: a,
        max_tokens_limit=4096
    )

    ### DIFFERENCES START HERE

    # Need to copy these otherwise various apps will feature templates that
    # point to the same objects.
    app.combine_docs_chain.llm_chain.prompt = \
        app.combine_docs_chain.llm_chain.prompt.copy()
    app.combine_docs_chain.document_prompt = \
        app.combine_docs_chain.document_prompt.copy()

    # Language mismatch fix via a prompt adjustment:
    app.combine_docs_chain.llm_chain.prompt.template = \
        "Use the following pieces of context to answer the question at the end " \
        "in the same language as the question. If you don't know the answer, " \
        "just say that you don't know, don't try to make up an answer.\n\n" \
        "{context}\n\n" \
        "Question: {question}\n" \
        "Helpful Answer: "

    ### END OF DIFFERENCES

    # Trulens instrumentation.
    tc = Tru().Chain(
        app_id=f"{app_id}/v2",
        chain=app,
        feedbacks=all_feedbacks,
        feedback_mode=feedback_mode
    )

    return tc

In [ ]:
# Instantiate the version 2 app:

tc2 = v2_new_conversation()

# Now the non-English question again:

res, record = tc2.call_with_record("Was ist QII?")
res

# Note that the response is now the appropriate language.

In [ ]:
# And the language match feedback is happy:

feedback = f_lang_match.run(record=record, app=tc2)
feedback.dict()

# TruBot Version 3: Context Filtering with Relevance

In [ ]:
def v3_new_conversation(feedback_mode=FeedbackMode.WITH_APP):
    """
    Create a langchain app for a new conversation with a question-answering bot.
    """

    # Blank conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=650,
        llm=llm,
        memory_key="chat_history",
        output_key='answer'
    )

    docsearch = get_doc_search()

    # Context retriever.
    retriever = docsearch.as_retriever()

    ### DIFFERENCES START HERE

    # Modified retriever that first filters returned contexts using
    # f_qs_relevance with a minimum relevance threshold (of 0.5):
    retriever_filtered = WithFeedbackFilterDocuments.of_retriever(
        retriever=retriever, feedback=f_qs_relevance, threshold = 0.5
    )

    ### END OF DIFFERENCES

    # Conversational app puts it all together.
    app = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever_filtered,
        return_source_documents=True,
        memory=memory,
        get_chat_history=lambda a: a,
        max_tokens_limit=4096
    )

    # Trulens instrumentation.
    tc = Tru().Chain(
        app_id=f"{app_id}/v3",
        chain=app,
        feedbacks=all_feedbacks,
        feedback_mode=feedback_mode
    )

    return tc

In [ ]:
# Instantiate the version 3 app:

tc3 = v3_new_conversation()

# Call the app:

res, record = tc3.call_with_record("Who is Shayak?")
res

# Notice the `source_documents` returned now does not include the low-relevance
# chunks and the answer likewise does not reference them.

# TruBot Version 4: Lang match fix and context filter

This is left as an exercise to the reader. 

In [ ]:
def v4_new_conversation(feedback_mode=FeedbackMode.WITH_APP):
    """
    Create a langchain app for a new conversation with a question-answering bot.
    """

    ### TO FILL IN HERE ###
    app = ...
    ### END OF TO FILL IN ###

    # Trulens instrumentation.
    tc = Tru().Chain(
        app_id=f"{app_id}/v4",
        chain=app,
        feedbacks=all_feedbacks,
        feedback_mode=feedback_mode
    )

    return tc

# Test conversations

Lets try out the 3 (or 4) trubot versions on a collection of test instances
about Shayak and some technical terms in several languages.

In [ ]:
apps = [
    v1_new_conversation,
    v2_new_conversation,
    v3_new_conversation,
    # v4_new_conversation # include this if you completed the exercise
]

questions = [
    "Who is Shayak?", "Wer ist Shayak?", "Kim jest Shayak?", "¿Quién es Shayak?", 
    "What is QII?", "Was ist QII?", "Co jest QII?", "¿Que es QII?"
]

# Comment out the next two lines to try all of the version and question
# combinations. Otherwise we select here only 2 questions and 2 models to start with.
apps = apps[0:2]
questions = questions[0:2]

def test_app_on_question(app, question):
    print(app.__name__, question)
    app = app(feedback_mode=FeedbackMode.DEFERRED)
    answer = app.call_with_record(question)
    return answer

# This asks all of the questions in parallel:
for app in apps:
    for question in questions:
        TP().promise(
            test_app_on_question,
            app=app,
            question=question
        )

TP().finish()

In [ ]:
# For deferred feedback evaluation. Start this:

Tru().start_evaluator(restart=True)